In [8]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

# You can reuse these functions from your original code
# Function to calculate the corresponding angle using the Ackermann relationship
def calculate_inner_outer_angles(turning_radius, wheelbase, track_width):
    """Calculate both inner and outer angles based on turning radius"""
    R = turning_radius
    L = wheelbase
    B = track_width
    
    inner_angle_rad = np.arctan(L / (R - B/2))
    outer_angle_rad = np.arctan(L / (R + B/2))
    
    return np.rad2deg(inner_angle_rad), np.rad2deg(outer_angle_rad)

def calculate_outer_angle(inner_angle, B, L):
    """Calculate outer angle from inner angle using Ackermann relationship"""
    inner_rad = np.deg2rad(inner_angle)
    cot_inner = 1 / np.tan(inner_rad)
    cot_outer = cot_inner + B/L
    if np.abs(cot_outer) < 1e-10:
        outer_rad = np.pi/2 if cot_outer >= 0 else -np.pi/2
    else:
        outer_rad = np.arctan(1 / cot_outer)
    return np.rad2deg(outer_rad)

def calculate_turning_radius(inner_angle, B, L):
    """Calculate turning radius to center of rear axle from inner angle"""
    inner_rad = np.deg2rad(inner_angle)
    Ri = L / np.tan(inner_rad)
    R = Ri + B/2
    return R

def calculate_ackermann_percentage(inner_angle, outer_angle, B, L):
    """Calculate how close the steering is to perfect Ackermann geometry"""
    ideal_outer = calculate_outer_angle(inner_angle, B, L)
    deviation = np.abs(ideal_outer - outer_angle)
    max_deviation = np.abs(ideal_outer - inner_angle)
    
    if max_deviation == 0:
        return 100.0
    
    return (1 - deviation / max_deviation) * 100.0

# Initial parameters
wheelbase = 2.5  # L (m)
track_width = 1.5  # B (m)
turning_radius = 5.0  # R (m) to outside wheel
max_steering_angle = 40.0  # Maximum steering angle in degrees
steering_ratio = 15.0  # Steering wheel to road wheel ratio
kingpin_inclination = 10.0  # Degrees
caster_angle = 5.0  # Degrees
scrub_radius = 0.05  # meters

# Calculate initial angles
inner_angle_init, outer_angle_init = calculate_inner_outer_angles(turning_radius, wheelbase, track_width)

# Create figure with subplots
fig = make_subplots(
    rows=3, cols=3,
    specs=[
        [{"type": "xy"}, {"type": "xy"}, {"type": "xy"}],
        [{"type": "xy"}, {"type": "xy"}, {"type": "xy"}],
        [{"colspan": 3, "type": "xy"}, None, None],
    ],
    subplot_titles=(
        "Ackermann Steering Angle Relationship", 
        "Turning Radius vs Inner Angle", 
        "Wheel Steering Geometry",
        "Ackermann Compliance", 
        "Steering Ratio Effects",
        "Instructions",
        "Vehicle Top-Down View"
    ),
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Prepare data for plots
inner_angles = np.linspace(1, 45, 100)  # degrees
outer_angles = [calculate_outer_angle(inner, track_width, wheelbase) for inner in inner_angles]
parallel_steering = inner_angles  # For comparison (parallel steering has equal angles)
turning_radii = [calculate_turning_radius(inner, track_width, wheelbase) for inner in inner_angles]
ackermann_pcts = [calculate_ackermann_percentage(inner, calculate_outer_angle(inner, track_width, wheelbase), 
                                          track_width, wheelbase) for inner in inner_angles]
steering_wheel_angles = np.linspace(0, 720, 100)  # Steering wheel angle (degrees)
road_wheel_angles = [min(angle/steering_ratio, max_steering_angle) for angle in steering_wheel_angles]

# Create the plots
# 1. Angle relationship plot
fig.add_trace(
    go.Scatter(x=inner_angles, y=outer_angles, mode='lines', name='Ackermann', line=dict(color='blue', width=2)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=inner_angles, y=parallel_steering, mode='lines', name='Parallel', 
               line=dict(color='black', width=2, dash='dash')),
    row=1, col=1
)
angle_marker = fig.add_trace(
    go.Scatter(x=[inner_angle_init], y=[outer_angle_init], mode='markers', 
               marker=dict(color='red', size=10), showlegend=False),
    row=1, col=1
)

# 2. Turning radius plot
fig.add_trace(
    go.Scatter(x=inner_angles, y=turning_radii, mode='lines', 
               line=dict(color='green', width=2), showlegend=False),
    row=1, col=2
)
radius_marker = fig.add_trace(
    go.Scatter(x=[inner_angle_init], y=[turning_radius], mode='markers', 
               marker=dict(color='red', size=10), showlegend=False),
    row=1, col=2
)

# 3. 3D wheel geometry placeholder
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='text', 
               text=["Interactive 3D wheel geometry<br>will be implemented with controls"],
               textposition="middle center",
               showlegend=False),
    row=1, col=3
)

# 4. Ackermann percentage plot
fig.add_trace(
    go.Scatter(x=inner_angles, y=ackermann_pcts, mode='lines', 
               line=dict(color='blue', width=2), showlegend=False),
    row=2, col=1
)
ackermann_marker = fig.add_trace(
    go.Scatter(x=[inner_angle_init], 
               y=[calculate_ackermann_percentage(inner_angle_init, outer_angle_init, track_width, wheelbase)], 
               mode='markers', marker=dict(color='red', size=10), showlegend=False),
    row=2, col=1
)

# 5. Steering ratio plot
fig.add_trace(
    go.Scatter(x=steering_wheel_angles, y=road_wheel_angles, mode='lines', 
               line=dict(color='magenta', width=2), showlegend=False),
    row=2, col=2
)
steer_ratio_marker = fig.add_trace(
    go.Scatter(x=[inner_angle_init*steering_ratio], y=[inner_angle_init], mode='markers', 
               marker=dict(color='red', size=10), showlegend=False),
    row=2, col=2
)

# 6. Instructions panel
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='text', 
               text=["<b>Ackermann Steering Simulator</b><br><br>"
                     "• Use widgets below to adjust vehicle parameters<br>"
                     "• Observe how changes affect steering geometry<br>"
                     "• Ackermann steering ensures inner wheel turns<br>"
                     "  at a sharper angle than the outer wheel<br>"
                     "• This reduces tire scrub during turns"],
               textposition="middle center",
               showlegend=False),
    row=2, col=3
)

# 7. Vehicle visualization (will be updated by the callback)
car_x = [0, wheelbase, wheelbase, 0, 0]
car_y = [-track_width/2, -track_width/2, track_width/2, track_width/2, -track_width/2]

# Draw vehicle body
fig.add_trace(
    go.Scatter(x=car_x, y=car_y, mode='lines', line=dict(color='blue', width=2), 
               name='Vehicle', fill='toself', fillcolor='rgba(0,0,255,0.1)'),
    row=3, col=1
)

# Add wheels (simplified rectangles for now)
wheel_length = wheelbase * 0.1
wheel_width = track_width * 0.1

# Function to create wheel coordinates
def get_wheel_coords(x_center, y_center, angle_deg):
    # Define wheel as a rectangle
    l, w = wheel_length, wheel_width
    points_x = [-l/2, l/2, l/2, -l/2, -l/2]
    points_y = [-w/2, -w/2, w/2, w/2, -w/2]
    
    # Rotate points
    angle_rad = np.deg2rad(angle_deg)
    rotated_x = []
    rotated_y = []
    for x, y in zip(points_x, points_y):
        rx = x * np.cos(angle_rad) - y * np.sin(angle_rad)
        ry = x * np.sin(angle_rad) + y * np.cos(angle_rad)
        rotated_x.append(rx + x_center)
        rotated_y.append(ry + y_center)
    
    return rotated_x, rotated_y

# Front left wheel (inner wheel)
fl_x, fl_y = get_wheel_coords(wheelbase, track_width/2, inner_angle_init)
fig.add_trace(
    go.Scatter(x=fl_x, y=fl_y, mode='lines', line=dict(color='black', width=2), 
               fill='toself', fillcolor='black', name='Front Left Wheel'),
    row=3, col=1
)

# Front right wheel (outer wheel)
fr_x, fr_y = get_wheel_coords(wheelbase, -track_width/2, outer_angle_init)
fig.add_trace(
    go.Scatter(x=fr_x, y=fr_y, mode='lines', line=dict(color='black', width=2), 
               fill='toself', fillcolor='black', name='Front Right Wheel'),
    row=3, col=1
)

# Rear left wheel
rl_x, rl_y = get_wheel_coords(0, track_width/2, 0)
fig.add_trace(
    go.Scatter(x=rl_x, y=rl_y, mode='lines', line=dict(color='black', width=2), 
               fill='toself', fillcolor='black', name='Rear Left Wheel'),
    row=3, col=1
)

# Rear right wheel
rr_x, rr_y = get_wheel_coords(0, -track_width/2, 0)
fig.add_trace(
    go.Scatter(x=rr_x, y=rr_y, mode='lines', line=dict(color='black', width=2), 
               fill='toself', fillcolor='black', name='Rear Right Wheel'),
    row=3, col=1
)

# Add turning circle
tc_x = -turning_radius
tc_y = 0

# Add turning center marker
fig.add_trace(
    go.Scatter(x=[tc_x], y=[tc_y], mode='markers', marker=dict(color='red', size=10),
               name='Turning Center'),
    row=3, col=1
)

# Add turning radius lines
fig.add_trace(
    go.Scatter(x=[tc_x, 0], y=[tc_y, 0], mode='lines', line=dict(color='red', width=1, dash='dash'),
               name='Radius to Rear Axle'),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=[tc_x, wheelbase], y=[tc_y, track_width/2], mode='lines', 
               line=dict(color='red', width=1, dash='dash'),
               name='Radius to Inner Wheel'),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=[tc_x, wheelbase], y=[tc_y, -track_width/2], mode='lines', 
               line=dict(color='red', width=1, dash='dash'),
               name='Radius to Outer Wheel'),
    row=3, col=1
)

# Add text information
info_text = f"Inner angle: {inner_angle_init:.1f}°<br>Outer angle: {outer_angle_init:.1f}°<br>"
info_text += f"Turning radius: {turning_radius:.1f}m<br>Track width: {track_width:.1f}m<br>Wheelbase: {wheelbase:.1f}m<br>"
info_text += f"Ackermann %: {calculate_ackermann_percentage(inner_angle_init, outer_angle_init, track_width, wheelbase):.1f}%"

fig.add_trace(
    go.Scatter(x=[wheelbase/2], y=[track_width], mode='text', 
               text=[info_text],
               textposition="top right",
               showlegend=False),
    row=3, col=1
)

# Update layout
fig.update_layout(
    height=900,
    width=1200,
    title_text="Ackermann Steering Simulator",
    showlegend=True,
    # Position the legend outside the plot area on the right side
    legend=dict(
        x=1.02,
        y=0.5,
        xanchor="left",
        yanchor="middle",
        bordercolor="Black",
        borderwidth=1,
        orientation="v"
    )
)

# Update axes
fig.update_xaxes(title_text="Inner Steering Angle (degrees)", range=[0, 45], row=1, col=1)
fig.update_yaxes(title_text="Outer Steering Angle (degrees)", range=[0, 45], row=1, col=1)

fig.update_xaxes(title_text="Inner Steering Angle (degrees)", range=[0, 45], row=1, col=2)
fig.update_yaxes(title_text="Turning Radius (m)", range=[0, 20], row=1, col=2)

fig.update_xaxes(title_text="Inner Steering Angle (degrees)", range=[0, 45], row=2, col=1)
fig.update_yaxes(title_text="Ackermann Percentage (%)", range=[0, 110], row=2, col=1)

fig.update_xaxes(title_text="Steering Wheel Angle (degrees)", range=[0, 720], row=2, col=2)
fig.update_yaxes(title_text="Road Wheel Angle (degrees)", range=[0, max_steering_angle*1.1], row=2, col=2)

fig.update_xaxes(title_text="X (m)", range=[-12, 5], row=3, col=1)
fig.update_yaxes(title_text="Y (m)", range=[-5, 5], row=3, col=1)

# Create interactive widgets
control_mode = widgets.RadioButtons(
    options=['Turning Radius', 'Inner Angle'],
    value='Turning Radius',
    description='Control Mode:',
    disabled=False
)

turning_radius_slider = widgets.FloatSlider(
    value=turning_radius,
    min=2.0,
    max=20.0,
    step=0.1,
    description='Turning Radius (m):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

inner_angle_slider = widgets.FloatSlider(
    value=inner_angle_init,
    min=1.0,
    max=45.0,
    step=0.1,
    description='Inner Angle (°):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

track_width_slider = widgets.FloatSlider(
    value=track_width,
    min=0.5,
    max=3.0,
    step=0.1,
    description='Track Width (m):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

wheelbase_slider = widgets.FloatSlider(
    value=wheelbase,
    min=1.0,
    max=5.0,
    step=0.1,
    description='Wheelbase (m):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

steering_ratio_slider = widgets.FloatSlider(
    value=steering_ratio,
    min=5.0,
    max=25.0,
    step=0.5,
    description='Steering Ratio:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

show_turning_circle = widgets.Checkbox(
    value=True,
    description='Show Turning Circle',
    disabled=False
)

show_steering_arms = widgets.Checkbox(
    value=False,
    description='Show Steering Arms',
    disabled=False
)

reset_button = widgets.Button(
    description='Reset Parameters',
    disabled=False,
    button_style='', 
    tooltip='Reset all parameters to default values',
    icon='refresh'
)

# Define update function for widgets
def update_plot(change=None):
    global wheelbase, track_width, turning_radius, steering_ratio
    
    # Get values from sliders
    wheelbase = wheelbase_slider.value
    track_width = track_width_slider.value
    steering_ratio = steering_ratio_slider.value
    
    # Handle control mode
    if control_mode.value == 'Turning Radius':
        turning_radius = turning_radius_slider.value
        inner_angle, outer_angle = calculate_inner_outer_angles(turning_radius, wheelbase, track_width)
        # Update inner angle slider without triggering callback
        inner_angle_slider.value = inner_angle
    else:  # 'Inner Angle'
        inner_angle = inner_angle_slider.value
        turning_radius = calculate_turning_radius(inner_angle, track_width, wheelbase)
        outer_angle = calculate_outer_angle(inner_angle, track_width, wheelbase)
        # Update turning radius slider without triggering callback
        turning_radius_slider.value = turning_radius
    
    # Recalculate all dependent values
    outer_angles = [calculate_outer_angle(inner, track_width, wheelbase) for inner in inner_angles]
    turning_radii = [calculate_turning_radius(inner, track_width, wheelbase) for inner in inner_angles]
    ackermann_pcts = [calculate_ackermann_percentage(inner, calculate_outer_angle(inner, track_width, wheelbase), 
                                              track_width, wheelbase) for inner in inner_angles]
    road_wheel_angles = [min(angle/steering_ratio, max_steering_angle) for angle in steering_wheel_angles]
    
    # Update angle relationship plot
    with fig.batch_update():
        fig.data[1].y = outer_angles  # Ackermann line
        fig.data[2].x = [inner_angle]  # Angle marker X
        fig.data[2].y = [outer_angle]  # Angle marker Y
        
        # Update turning radius plot
        fig.data[3].y = turning_radii  # Turning radius line
        fig.data[4].x = [inner_angle]  # Radius marker X
        fig.data[4].y = [turning_radius]  # Radius marker Y
        
        # Update Ackermann percentage plot
        fig.data[6].y = ackermann_pcts  # Ackermann percentage line
        current_ackermann_pct = calculate_ackermann_percentage(inner_angle, outer_angle, track_width, wheelbase)
        fig.data[7].x = [inner_angle]  # Ackermann marker X
        fig.data[7].y = [current_ackermann_pct]  # Ackermann marker Y
        
        # Update steering ratio plot
        fig.data[8].y = road_wheel_angles  # Steering ratio line
        fig.data[9].x = [inner_angle*steering_ratio]  # Steering ratio marker X
        fig.data[9].y = [inner_angle]  # Steering ratio marker Y
        
        # Update vehicle visualization
        # Update car body
        car_x = [0, wheelbase, wheelbase, 0, 0]
        car_y = [-track_width/2, -track_width/2, track_width/2, track_width/2, -track_width/2]
        fig.data[11].x = car_x
        fig.data[11].y = car_y
        
        # Update wheels
        # Front left wheel (inner wheel)
        fl_x, fl_y = get_wheel_coords(wheelbase, track_width/2, inner_angle)
        fig.data[12].x = fl_x
        fig.data[12].y = fl_y
        
        # Front right wheel (outer wheel)
        fr_x, fr_y = get_wheel_coords(wheelbase, -track_width/2, outer_angle)
        fig.data[13].x = fr_x
        fig.data[13].y = fr_y
        
        # Rear left wheel
        rl_x, rl_y = get_wheel_coords(0, track_width/2, 0)
        fig.data[14].x = rl_x
        fig.data[14].y = rl_y
        
        # Rear right wheel
        rr_x, rr_y = get_wheel_coords(0, -track_width/2, 0)
        fig.data[15].x = rr_x
        fig.data[15].y = rr_y
        
        # Update turning center and radius
        tc_x = -turning_radius
        tc_y = 0
        
        # Update turning center marker
        fig.data[16].x = [tc_x]
        fig.data[16].y = [tc_y]
        
        # Update turning radius lines
        fig.data[17].x = [tc_x, 0]  # To rear axle center
        fig.data[17].y = [tc_y, 0]
        
        fig.data[18].x = [tc_x, wheelbase]  # To inner wheel
        fig.data[18].y = [tc_y, track_width/2]
        
        fig.data[19].x = [tc_x, wheelbase]  # To outer wheel
        fig.data[19].y = [tc_y, -track_width/2]
        
        # Update text information
        info_text = f"Inner angle: {inner_angle:.1f}°<br>Outer angle: {outer_angle:.1f}°<br>"
        info_text += f"Turning radius: {turning_radius:.1f}m<br>Track width: {track_width:.1f}m<br>Wheelbase: {wheelbase:.1f}m<br>"
        info_text += f"Ackermann %: {current_ackermann_pct:.1f}%"
        fig.data[20].text = [info_text]
        
        # Update axes ranges for vehicle plot
        fig.update_xaxes(range=[-turning_radius-1, wheelbase+1], row=3, col=1)
        fig.update_yaxes(range=[-max(track_width, turning_radius/2)-1, max(track_width, turning_radius/2)+1], row=3, col=1)

def reset_parameters(b):
    global wheelbase, track_width, turning_radius, steering_ratio
    
    # Reset to initial values
    wheelbase_slider.value = 2.5
    track_width_slider.value = 1.5
    turning_radius_slider.value = 5.0
    steering_ratio_slider.value = 15.0
    
    # Force update
    update_plot()

# Connect callbacks
control_mode.observe(update_plot, 'value')
turning_radius_slider.observe(update_plot, 'value')
inner_angle_slider.observe(update_plot, 'value')
track_width_slider.observe(update_plot, 'value')
wheelbase_slider.observe(update_plot, 'value')
steering_ratio_slider.observe(update_plot, 'value')
show_turning_circle.observe(update_plot, 'value')
show_steering_arms.observe(update_plot, 'value')
reset_button.on_click(reset_parameters)

# Display the widgets and figure
controls1 = widgets.HBox([control_mode, reset_button])
controls2 = widgets.HBox([turning_radius_slider, inner_angle_slider])
controls3 = widgets.HBox([track_width_slider, wheelbase_slider])
controls4 = widgets.HBox([steering_ratio_slider, widgets.HBox([show_turning_circle, show_steering_arms])])

controls = widgets.VBox([controls1, controls2, controls3, controls4])
display(controls)
fig.show()

In [9]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
from scipy.optimize import fsolve
from math import sin, cos, radians, degrees, sqrt, tan, atan


# Function to calculate the corresponding angle using the Ackermann relationship
def calculate_inner_outer_angles(turning_radius, wheelbase, track_width):
    """Calculate both inner and outer angles based on turning radius"""
    R = turning_radius
    L = wheelbase
    B = track_width
    
    inner_angle_rad = np.arctan(L / (R - B/2))
    outer_angle_rad = np.arctan(L / (R + B/2))
    
    return np.rad2deg(inner_angle_rad), np.rad2deg(outer_angle_rad)

def calculate_outer_angle(inner_angle, B, L):
    """Calculate outer angle from inner angle using Ackermann relationship"""
    inner_rad = np.deg2rad(inner_angle)
    cot_inner = 1 / np.tan(inner_rad)
    cot_outer = cot_inner + B/L
    if np.abs(cot_outer) < 1e-10:
        outer_rad = np.pi/2 if cot_outer >= 0 else -np.pi/2
    else:
        outer_rad = np.arctan(1 / cot_outer)
    return np.rad2deg(outer_rad)

def calculate_turning_radius(inner_angle, B, L):
    """Calculate turning radius to center of rear axle from inner angle"""
    inner_rad = np.deg2rad(inner_angle)
    Ri = L / np.tan(inner_rad)
    R = Ri + B/2
    return R

def calculate_ackermann_percentage(inner_angle, outer_angle, B, L):
    """Calculate how close the steering is to perfect Ackermann geometry"""
    ideal_outer = calculate_outer_angle(inner_angle, B, L)
    deviation = np.abs(ideal_outer - outer_angle)
    max_deviation = np.abs(ideal_outer - inner_angle)
    
    if max_deviation == 0:
        return 100.0
    
    return (1 - deviation / max_deviation) * 100.0
# Initial parameters
wheelbase = 2.5  # L (m)
track_width = 1.5  # B (m)
turning_radius = 5.0  # R (m) to outside wheel
max_steering_angle = 40.0  # Maximum steering angle in degrees
steering_ratio = 15.0  # Steering wheel to road wheel ratio
kingpin_inclination = 10.0  # Degrees
caster_angle = 5.0  # Degrees
scrub_radius = 0.05  # meters

# Calculate initial angles
inner_angle_init, outer_angle_init = calculate_inner_outer_angles(turning_radius, wheelbase, track_width)

# Create figure with subplots
fig = make_subplots(
    rows=3, cols=3,
    specs=[
        [{"type": "xy"}, {"type": "xy"}, {"type": "xy"}],
        [{"type": "xy"}, {"type": "xy"}, {"type": "xy"}],
        [{"colspan": 3, "type": "xy"}, None, None],
    ],
    subplot_titles=(
        "Ackermann Steering Angle Relationship", 
        "Turning Radius vs Inner Angle", 
        "Wheel Steering Geometry",
        "Ackermann Compliance", 
        "Steering Ratio Effects",
        "Instructions",
        "Vehicle Top-Down View"
    ),
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Prepare data for plots
inner_angles = np.linspace(1, 45, 100)  # degrees
outer_angles = [calculate_outer_angle(inner, track_width, wheelbase) for inner in inner_angles]
parallel_steering = inner_angles  # For comparison (parallel steering has equal angles)
turning_radii = [calculate_turning_radius(inner, track_width, wheelbase) for inner in inner_angles]
ackermann_pcts = [calculate_ackermann_percentage(inner, calculate_outer_angle(inner, track_width, wheelbase), 
                                          track_width, wheelbase) for inner in inner_angles]
steering_wheel_angles = np.linspace(0, 720, 100)  # Steering wheel angle (degrees)
road_wheel_angles = [min(angle/steering_ratio, max_steering_angle) for angle in steering_wheel_angles]

# Create the plots
# 1. Angle relationship plot
fig.add_trace(
    go.Scatter(x=inner_angles, y=outer_angles, mode='lines', name='Ackermann', line=dict(color='blue', width=2)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=inner_angles, y=parallel_steering, mode='lines', name='Parallel', 
               line=dict(color='black', width=2, dash='dash')),
    row=1, col=1
)
angle_marker = fig.add_trace(
    go.Scatter(x=[inner_angle_init], y=[outer_angle_init], mode='markers', 
               marker=dict(color='red', size=10), showlegend=False),
    row=1, col=1
)

# 2. Turning radius plot
fig.add_trace(
    go.Scatter(x=inner_angles, y=turning_radii, mode='lines', 
               line=dict(color='green', width=2), showlegend=False),
    row=1, col=2
)
radius_marker = fig.add_trace(
    go.Scatter(x=[inner_angle_init], y=[turning_radius], mode='markers', 
               marker=dict(color='red', size=10), showlegend=False),
    row=1, col=2
)

# 3. 3D wheel geometry placeholder
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='text', 
               text=["Interactive 3D wheel geometry<br>will be implemented with controls"],
               textposition="middle center",
               showlegend=False),
    row=1, col=3
)

# 4. Ackermann percentage plot
fig.add_trace(
    go.Scatter(x=inner_angles, y=ackermann_pcts, mode='lines', 
               line=dict(color='blue', width=2), showlegend=False),
    row=2, col=1
)
ackermann_marker = fig.add_trace(
    go.Scatter(x=[inner_angle_init], 
               y=[calculate_ackermann_percentage(inner_angle_init, outer_angle_init, track_width, wheelbase)], 
               mode='markers', marker=dict(color='red', size=10), showlegend=False),
    row=2, col=1
)

# 5. Steering ratio plot
fig.add_trace(
    go.Scatter(x=steering_wheel_angles, y=road_wheel_angles, mode='lines', 
               line=dict(color='magenta', width=2), showlegend=False),
    row=2, col=2
)
steer_ratio_marker = fig.add_trace(
    go.Scatter(x=[inner_angle_init*steering_ratio], y=[inner_angle_init], mode='markers', 
               marker=dict(color='red', size=10), showlegend=False),
    row=2, col=2
)

# 6. Instructions panel
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='text', 
               text=["<b>Ackermann Steering Simulator</b><br><br>"
                     "• Use widgets below to adjust vehicle parameters<br>"
                     "• Observe how changes affect steering geometry<br>"
                     "• Ackermann steering ensures inner wheel turns<br>"
                     "  at a sharper angle than the outer wheel<br>"
                     "• This reduces tire scrub during turns"],
               textposition="middle center",
               showlegend=False),
    row=2, col=3
)

# 7. Vehicle visualization (will be updated by the callback)
car_x = [0, wheelbase, wheelbase, 0, 0]
car_y = [-track_width/2, -track_width/2, track_width/2, track_width/2, -track_width/2]

# Draw vehicle body
fig.add_trace(
    go.Scatter(x=car_x, y=car_y, mode='lines', line=dict(color='blue', width=2), 
               name='Vehicle', fill='toself', fillcolor='rgba(0,0,255,0.1)'),
    row=3, col=1
)

# Add wheels (simplified rectangles for now)
wheel_length = wheelbase * 0.1
wheel_width = track_width * 0.1

# Function to create wheel coordinates
def get_wheel_coords(x_center, y_center, angle_deg):
    # Define wheel as a rectangle
    l, w = wheel_length, wheel_width
    points_x = [-l/2, l/2, l/2, -l/2, -l/2]
    points_y = [-w/2, -w/2, w/2, w/2, -w/2]
    
    # Rotate points
    angle_rad = np.deg2rad(angle_deg)
    rotated_x = []
    rotated_y = []
    for x, y in zip(points_x, points_y):
        rx = x * np.cos(angle_rad) - y * np.sin(angle_rad)
        ry = x * np.sin(angle_rad) + y * np.cos(angle_rad)
        rotated_x.append(rx + x_center)
        rotated_y.append(ry + y_center)
    
    return rotated_x, rotated_y

# Front left wheel (inner wheel)
fl_x, fl_y = get_wheel_coords(wheelbase, track_width/2, inner_angle_init)
fig.add_trace(
    go.Scatter(x=fl_x, y=fl_y, mode='lines', line=dict(color='black', width=2), 
               fill='toself', fillcolor='black', name='Front Left Wheel'),
    row=3, col=1
)

# Front right wheel (outer wheel)
fr_x, fr_y = get_wheel_coords(wheelbase, -track_width/2, outer_angle_init)
fig.add_trace(
    go.Scatter(x=fr_x, y=fr_y, mode='lines', line=dict(color='black', width=2), 
               fill='toself', fillcolor='black', name='Front Right Wheel'),
    row=3, col=1
)

# Rear left wheel
rl_x, rl_y = get_wheel_coords(0, track_width/2, 0)
fig.add_trace(
    go.Scatter(x=rl_x, y=rl_y, mode='lines', line=dict(color='black', width=2), 
               fill='toself', fillcolor='black', name='Rear Left Wheel'),
    row=3, col=1
)

# Rear right wheel
rr_x, rr_y = get_wheel_coords(0, -track_width/2, 0)
fig.add_trace(
    go.Scatter(x=rr_x, y=rr_y, mode='lines', line=dict(color='black', width=2), 
               fill='toself', fillcolor='black', name='Rear Right Wheel'),
    row=3, col=1
)

# Add turning circle
tc_x = -turning_radius
tc_y = 0

# Add turning center marker
fig.add_trace(
    go.Scatter(x=[tc_x], y=[tc_y], mode='markers', marker=dict(color='red', size=10),
               name='Turning Center'),
    row=3, col=1
)

# Add turning radius lines
fig.add_trace(
    go.Scatter(x=[tc_x, 0], y=[tc_y, 0], mode='lines', line=dict(color='red', width=1, dash='dash'),
               name='Radius to Rear Axle'),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=[tc_x, wheelbase], y=[tc_y, track_width/2], mode='lines', 
               line=dict(color='red', width=1, dash='dash'),
               name='Radius to Inner Wheel'),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=[tc_x, wheelbase], y=[tc_y, -track_width/2], mode='lines', 
               line=dict(color='red', width=1, dash='dash'),
               name='Radius to Outer Wheel'),
    row=3, col=1
)

# Add text information
info_text = f"Inner angle: {inner_angle_init:.1f}°<br>Outer angle: {outer_angle_init:.1f}°<br>"
info_text += f"Turning radius: {turning_radius:.1f}m<br>Track width: {track_width:.1f}m<br>Wheelbase: {wheelbase:.1f}m<br>"
info_text += f"Ackermann %: {calculate_ackermann_percentage(inner_angle_init, outer_angle_init, track_width, wheelbase):.1f}%"

fig.add_trace(
    go.Scatter(x=[wheelbase/2], y=[track_width], mode='text', 
               text=[info_text],
               textposition="top right",
               showlegend=False),
    row=3, col=1
)

# Update layout
fig.update_layout(
    height=900,
    width=1200,
    title_text="Ackermann Steering Simulator",
    showlegend=True,
    # Position the legend outside the plot area on the right side
    legend=dict(
        x=1.02,
        y=0.5,
        xanchor="left",
        yanchor="middle",
        bordercolor="Black",
        borderwidth=1,
        orientation="v"
    )
)

# Update axes
fig.update_xaxes(title_text="Inner Steering Angle (degrees)", range=[0, 45], row=1, col=1)
fig.update_yaxes(title_text="Outer Steering Angle (degrees)", range=[0, 45], row=1, col=1)

fig.update_xaxes(title_text="Inner Steering Angle (degrees)", range=[0, 45], row=1, col=2)
fig.update_yaxes(title_text="Turning Radius (m)", range=[0, 20], row=1, col=2)

fig.update_xaxes(title_text="Inner Steering Angle (degrees)", range=[0, 45], row=2, col=1)
fig.update_yaxes(title_text="Ackermann Percentage (%)", range=[0, 110], row=2, col=1)

fig.update_xaxes(title_text="Steering Wheel Angle (degrees)", range=[0, 720], row=2, col=2)
fig.update_yaxes(title_text="Road Wheel Angle (degrees)", range=[0, max_steering_angle*1.1], row=2, col=2)

fig.update_xaxes(title_text="X (m)", range=[-12, 5], row=3, col=1)
fig.update_yaxes(title_text="Y (m)", range=[-5, 5], row=3, col=1)

# Create interactive widgets
control_mode = widgets.RadioButtons(
    options=['Turning Radius', 'Inner Angle'],
    value='Turning Radius',
    description='Control Mode:',
    disabled=False
)

turning_radius_slider = widgets.FloatSlider(
    value=turning_radius,
    min=2.0,
    max=20.0,
    step=0.1,
    description='Turning Radius (m):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

inner_angle_slider = widgets.FloatSlider(
    value=inner_angle_init,
    min=1.0,
    max=45.0,
    step=0.1,
    description='Inner Angle (°):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

track_width_slider = widgets.FloatSlider(
    value=track_width,
    min=0.5,
    max=3.0,
    step=0.1,
    description='Track Width (m):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

wheelbase_slider = widgets.FloatSlider(
    value=wheelbase,
    min=1.0,
    max=5.0,
    step=0.1,
    description='Wheelbase (m):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

steering_ratio_slider = widgets.FloatSlider(
    value=steering_ratio,
    min=5.0,
    max=25.0,
    step=0.5,
    description='Steering Ratio:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

show_turning_circle = widgets.Checkbox(
    value=True,
    description='Show Turning Circle',
    disabled=False
)

show_steering_arms = widgets.Checkbox(
    value=False,
    description='Show Steering Arms',
    disabled=False
)

reset_button = widgets.Button(
    description='Reset Parameters',
    disabled=False,
    button_style='', 
    tooltip='Reset all parameters to default values',
    icon='refresh'
)

# Define update function for widgets
def update_plot(change=None):
    global wheelbase, track_width, turning_radius, steering_ratio
    
    # Get values from sliders
    wheelbase = wheelbase_slider.value
    track_width = track_width_slider.value
    steering_ratio = steering_ratio_slider.value
    
    # Handle control mode
    if control_mode.value == 'Turning Radius':
        turning_radius = turning_radius_slider.value
        inner_angle, outer_angle = calculate_inner_outer_angles(turning_radius, wheelbase, track_width)
        # Update inner angle slider without triggering callback
        inner_angle_slider.value = inner_angle
    else:  # 'Inner Angle'
        inner_angle = inner_angle_slider.value
        turning_radius = calculate_turning_radius(inner_angle, track_width, wheelbase)
        outer_angle = calculate_outer_angle(inner_angle, track_width, wheelbase)
        # Update turning radius slider without triggering callback
        turning_radius_slider.value = turning_radius
    
    # Recalculate all dependent values
    outer_angles = [calculate_outer_angle(inner, track_width, wheelbase) for inner in inner_angles]
    turning_radii = [calculate_turning_radius(inner, track_width, wheelbase) for inner in inner_angles]
    ackermann_pcts = [calculate_ackermann_percentage(inner, calculate_outer_angle(inner, track_width, wheelbase), 
                                              track_width, wheelbase) for inner in inner_angles]
    road_wheel_angles = [min(angle/steering_ratio, max_steering_angle) for angle in steering_wheel_angles]
    
    # Update angle relationship plot
    with fig.batch_update():
        fig.data[1].y = outer_angles  # Ackermann line
        fig.data[2].x = [inner_angle]  # Angle marker X
        fig.data[2].y = [outer_angle]  # Angle marker Y
        
        # Update turning radius plot
        fig.data[3].y = turning_radii  # Turning radius line
        fig.data[4].x = [inner_angle]  # Radius marker X
        fig.data[4].y = [turning_radius]  # Radius marker Y
        
        # Update Ackermann percentage plot
        fig.data[6].y = ackermann_pcts  # Ackermann percentage line
        current_ackermann_pct = calculate_ackermann_percentage(inner_angle, outer_angle, track_width, wheelbase)
        fig.data[7].x = [inner_angle]  # Ackermann marker X
        fig.data[7].y = [current_ackermann_pct]  # Ackermann marker Y
        
        # Update steering ratio plot
        fig.data[8].y = road_wheel_angles  # Steering ratio line
        fig.data[9].x = [inner_angle*steering_ratio]  # Steering ratio marker X
        fig.data[9].y = [inner_angle]  # Steering ratio marker Y
        
        # Update vehicle visualization
        # Update car body
        car_x = [0, wheelbase, wheelbase, 0, 0]
        car_y = [-track_width/2, -track_width/2, track_width/2, track_width/2, -track_width/2]
        fig.data[11].x = car_x
        fig.data[11].y = car_y
        
        # Update wheels
        # Front left wheel (inner wheel)
        fl_x, fl_y = get_wheel_coords(wheelbase, track_width/2, inner_angle)
        fig.data[12].x = fl_x
        fig.data[12].y = fl_y
        
        # Front right wheel (outer wheel)
        fr_x, fr_y = get_wheel_coords(wheelbase, -track_width/2, outer_angle)
        fig.data[13].x = fr_x
        fig.data[13].y = fr_y
        
        # Rear left wheel
        rl_x, rl_y = get_wheel_coords(0, track_width/2, 0)
        fig.data[14].x = rl_x
        fig.data[14].y = rl_y
        
        # Rear right wheel
        rr_x, rr_y = get_wheel_coords(0, -track_width/2, 0)
        fig.data[15].x = rr_x
        fig.data[15].y = rr_y
        
        # Update turning center and radius
        tc_x = -turning_radius
        tc_y = 0
        
        # Update turning center marker
        fig.data[16].x = [tc_x]
        fig.data[16].y = [tc_y]
        
        # Update turning radius lines
        fig.data[17].x = [tc_x, 0]  # To rear axle center
        fig.data[17].y = [tc_y, 0]
        
        fig.data[18].x = [tc_x, wheelbase]  # To inner wheel
        fig.data[18].y = [tc_y, track_width/2]
        
        fig.data[19].x = [tc_x, wheelbase]  # To outer wheel
        fig.data[19].y = [tc_y, -track_width/2]
        
        # Update text information
        info_text = f"Inner angle: {inner_angle:.1f}°<br>Outer angle: {outer_angle:.1f}°<br>"
        info_text += f"Turning radius: {turning_radius:.1f}m<br>Track width: {track_width:.1f}m<br>Wheelbase: {wheelbase:.1f}m<br>"
        info_text += f"Ackermann %: {current_ackermann_pct:.1f}%"
        fig.data[20].text = [info_text]
        
        # Update axes ranges for vehicle plot
        fig.update_xaxes(range=[-turning_radius-1, wheelbase+1], row=3, col=1)
        fig.update_yaxes(range=[-max(track_width, turning_radius/2)-1, max(track_width, turning_radius/2)+1], row=3, col=1)

def reset_parameters(b):
    global wheelbase, track_width, turning_radius, steering_ratio
    
    # Reset to initial values
    wheelbase_slider.value = 2.5
    track_width_slider.value = 1.5
    turning_radius_slider.value = 5.0
    steering_ratio_slider.value = 15.0
    
    # Force update
    update_plot()

# Connect callbacks
control_mode.observe(update_plot, 'value')
turning_radius_slider.observe(update_plot, 'value')
inner_angle_slider.observe(update_plot, 'value')
track_width_slider.observe(update_plot, 'value')
wheelbase_slider.observe(update_plot, 'value')
steering_ratio_slider.observe(update_plot, 'value')
show_turning_circle.observe(update_plot, 'value')
show_steering_arms.observe(update_plot, 'value')
reset_button.on_click(reset_parameters)

def create_wheel_geometry_fig(kingpin_angle, caster_angle, scrub_radius, steer_angle):
    """Create a 3D figure showing wheel geometry with steering parameters"""
    # Create a new 3D figure
    wheel_fig = go.Figure()
    
    # Convert angles to radians
    kingpin_rad = np.deg2rad(kingpin_angle)
    caster_rad = np.deg2rad(caster_angle)
    steer_rad = np.deg2rad(steer_angle)
    
    # Wheel dimensions
    wheel_radius = 0.3
    wheel_width = 0.2
    
    # Wheel center position (at the ground)
    wheel_x = 0
    wheel_y = 0
    wheel_z = wheel_radius
    
    # Create points for the wheel (circle in XZ plane)
    theta = np.linspace(0, 2*np.pi, 100)
    rim_x = wheel_x + wheel_radius * np.cos(theta)
    rim_y = np.zeros_like(theta) + wheel_y
    rim_z = wheel_z + wheel_radius * np.sin(theta)
    
    # Create points for the wheel width (side profile)
    side_y = np.linspace(-wheel_width/2, wheel_width/2, 2)
    side_x = np.zeros_like(side_y) + wheel_x
    side_z = np.zeros_like(side_y) + wheel_z
    
    # Rotate the wheel based on steering angle (around Z axis)
    rotated_rim_x = rim_x * np.cos(steer_rad) - rim_y * np.sin(steer_rad)
    rotated_rim_y = rim_x * np.sin(steer_rad) + rim_y * np.cos(steer_rad)
    
    rotated_side_x = side_x * np.cos(steer_rad) - side_y * np.sin(steer_rad)
    rotated_side_y = side_x * np.sin(steer_rad) + side_y * np.cos(steer_rad)
    
    # Add the wheel rim and side profile
    wheel_fig.add_trace(
        go.Scatter3d(
            x=rotated_rim_x, y=rotated_rim_y, z=rim_z,
            mode='lines',
            line=dict(color='black', width=6),
            name='Wheel Rim'
        )
    )
    
    # Define kingpin axis
    # Start at the top of the suspension (above wheel center)
    kingpin_top_x = 0
    kingpin_top_y = 0
    kingpin_top_z = wheel_radius * 2
    
    # Kingpin axis points down and inward (positive Y is inward in this model)
    # Length of the kingpin axis
    kingpin_length = wheel_radius * 2.2
    
    # Calculate bottom point of kingpin axis
    kingpin_bottom_x = kingpin_top_x + kingpin_length * np.sin(kingpin_rad) * np.sin(caster_rad)
    kingpin_bottom_y = kingpin_top_y + kingpin_length * np.sin(kingpin_rad) * np.cos(caster_rad)
    kingpin_bottom_z = kingpin_top_z - kingpin_length * np.cos(kingpin_rad)
    
    # Create kingpin axis
    wheel_fig.add_trace(
        go.Scatter3d(
            x=[kingpin_top_x, kingpin_bottom_x],
            y=[kingpin_top_y, kingpin_bottom_y],
            z=[kingpin_top_z, kingpin_bottom_z],
            mode='lines',
            line=dict(color='red', width=5),
            name='Kingpin Axis'
        )
    )
    
    # Add steering axis
    wheel_fig.add_trace(
        go.Scatter3d(
            x=[0, 0],
            y=[0, 0],
            z=[0, wheel_radius * 2],
            mode='lines',
            line=dict(color='blue', width=5, dash='dash'),
            name='Steering Axis'
        )
    )
    
    # Add scrub radius visualization
    contact_point_x = wheel_radius * np.sin(steer_rad)
    contact_point_y = wheel_radius * np.cos(steer_rad)
    contact_point_z = 0
    
    # Where kingpin axis intersects the ground
    kingpin_ground_x = kingpin_bottom_x
    kingpin_ground_y = kingpin_bottom_y
    kingpin_ground_z = 0
    
    # Add contact point
    wheel_fig.add_trace(
        go.Scatter3d(
            x=[contact_point_x],
            y=[contact_point_y],
            z=[contact_point_z],
            mode='markers',
            marker=dict(color='green', size=8),
            name='Contact Point'
        )
    )
    
    # Add kingpin intersection with ground
    wheel_fig.add_trace(
        go.Scatter3d(
            x=[kingpin_ground_x],
            y=[kingpin_ground_y],
            z=[kingpin_ground_z],
            mode='markers',
            marker=dict(color='orange', size=8),
            name='Kingpin Ground Point'
        )
    )
    
    # Add scrub radius line
    wheel_fig.add_trace(
        go.Scatter3d(
            x=[contact_point_x, kingpin_ground_x],
            y=[contact_point_y, kingpin_ground_y],
            z=[contact_point_z, kingpin_ground_z],
            mode='lines',
            line=dict(color='purple', width=4),
            name='Scrub Radius'
        )
    )
    
    # Add caster offset visualization
    wheel_fig.add_trace(
        go.Scatter3d(
            x=[kingpin_top_x, kingpin_top_x],
            y=[kingpin_top_y, kingpin_bottom_y],
            z=[kingpin_top_z, kingpin_top_z],
            mode='lines',
            line=dict(color='orange', width=4, dash='dot'),
            name='Caster Offset'
        )
    )
    
    # Configure the layout
    wheel_fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='data'
        ),
        margin=dict(l=0, r=0, b=0, t=30),
        scene_camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.0)
        ),
        title="3D Wheel Geometry"
    )
    
    return wheel_fig

# Create initial 3D wheel geometry
wheel_3d_fig = create_wheel_geometry_fig(kingpin_inclination, caster_angle, scrub_radius, inner_angle_init)

# Replace the placeholder with the 3D wheel geometry in the subplot
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='text', 
               text=["Loading 3D wheel geometry..."],
               textposition="middle center",
               showlegend=False),
    row=1, col=3
)

# Update the update_plot function to update the 3D wheel geometry
def update_plot(change=None):
    global wheelbase, track_width, turning_radius, steering_ratio, wheel_3d_fig
    
    # Get values from sliders
    wheelbase = wheelbase_slider.value
    track_width = track_width_slider.value
    steering_ratio = steering_ratio_slider.value
    
    # Keep existing code here...
    
    # After updating all the other plots, update the 3D wheel geometry
    # Create new 3D wheel geometry with updated angles
    wheel_3d_fig = create_wheel_geometry_fig(kingpin_inclination, caster_angle, scrub_radius, inner_angle)
    
    # The rest of your existing update_plot function...

# Add new sliders for wheel geometry parameters
kingpin_slider = widgets.FloatSlider(
    value=kingpin_inclination,
    min=0.0,
    max=20.0,
    step=0.5,
    description='Kingpin Incl. (°):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

caster_slider = widgets.FloatSlider(
    value=caster_angle,
    min=0.0,
    max=15.0,
    step=0.5,
    description='Caster Angle (°):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

scrub_radius_slider = widgets.FloatSlider(
    value=scrub_radius,
    min=-0.1,
    max=0.2,
    step=0.01,
    description='Scrub Radius (m):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# Update the wheel geometry when sliders change
kingpin_slider.observe(update_plot, 'value')
caster_slider.observe(update_plot, 'value')
scrub_radius_slider.observe(update_plot, 'value')

# Update the controls display to include the new sliders
# Add after the existing controls
controls5 = widgets.HBox([kingpin_slider, caster_slider, scrub_radius_slider])

# Update the controls VBox to include the new row
controls = widgets.VBox([controls1, controls2, controls3, controls4, controls5])

# Add a button to open the 3D view in a new window
view_3d_button = widgets.Button(
    description='Open 3D View',
    disabled=False,
    button_style='info',
    tooltip='Open 3D wheel geometry in a separate window',
    icon='cube'
)

def open_3d_view(b):
    """Open the 3D wheel geometry in a new window"""
    # Create a new figure with current parameters
    new_fig = create_wheel_geometry_fig(kingpin_slider.value, caster_slider.value, 
                                        scrub_radius_slider.value, inner_angle_slider.value)
    new_fig.update_layout(height=600, width=800)
    new_fig.show()

view_3d_button.on_click(open_3d_view)



# Add the button to controls1
controls1 = widgets.HBox([control_mode, reset_button, view_3d_button])
controls2 = widgets.HBox([turning_radius_slider, inner_angle_slider])
controls3 = widgets.HBox([track_width_slider, wheelbase_slider])
controls4 = widgets.HBox([steering_ratio_slider, widgets.HBox([show_turning_circle, show_steering_arms])])

controls = widgets.VBox([controls1, controls2, controls3, controls4])
display(controls)
fig.show()